In [1]:
# Import libaries

from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [2]:
final_list = []

# Skytrax URL
url = 'https://www.airlinequality.com/airline-reviews/jeju-air/?sortby=post_date%3ADesc&pagesize=20'

# Request URL
response = requests.get(url)

# Create Soup
soup = BeautifulSoup(response.content, 'lxml')

# For loop for HTML
for idx, review in enumerate(soup.find_all('div',{'class':"review-stats"})):
    
    # Create a empty dictionary to contain columns and values
    review_dict = dict()
    
    # Values that is in star fill
    names = review.find_all('td', class_='review-rating-header')[::-1]
    ratings = review.find_all('td', class_='stars')[::-1]
    
    for name, rating in zip(names[1:], ratings):
        review_dict[name.text] = rating.find_all(class_='fill')[-1].text
    
    # Author, Date, Overall_rating,Text_content, Recommended
    review_dict['Author'] = soup.find_all(itemprop='name')[idx+1].text
    review_dict['Date'] = soup.find_all(itemprop='datePublished')[::2][idx].attrs['content']
    review_dict['Overall_rating'] = soup.find_all('div',{'itemprop':'reviewRating'})[idx].find(itemprop="ratingValue").text
    review_dict['text_content'] = soup.find_all('div',class_='text_content')[idx].text
    review_dict['Recommended'] = soup.find_all('div',{'class':"review-stats"})[idx].find_all('tr')[-1].find_all('td')[-1].text
        
    # Type of Traveller
    try:
        review.find('td',{'class':'review-rating-header type_of_traveller'}).text is not None
        review_dict['traveller_type'] = soup.find_all('td',{'class':'review-rating-header type_of_traveller'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['traveller_type'] = np.nan
    
    # Seat type
    try:
        review.find('td',{'class':'review-rating-header cabin_flown'}).text is not None
        review_dict['seat_type'] = soup.find_all('td',{'class':'review-rating-header cabin_flown'})[idx].fetchNextSiblings()[0].text
    
    except:
        review_dict['seat_type'] = np.nan
    
    
    # Append dictionary into a list
    final_list.append(review_dict)

# Create DataFrame
df = pd.DataFrame(final_list)
df['airline'] = 'jeju air'

# rearrange dataframe
df = df[['airline','Author','Date','text_content','traveller_type','seat_type','Overall_rating','Seat Comfort',
 'Cabin Staff Service','Food & Beverages','Inflight Entertainment','Ground Service','Value For Money',  
  'Recommended']]

# rename dataframe
df.columns = ['airline_name', 'author', 'date', 'content', 'type_traveller','cabin_flown', 'overall_rating', 'seat_comfort_rating',
'cabin_staff_rating', 'food_beverages_rating','inflight_entertainment_rating', 'ground_service_rating','value_money_rating', 'recommended']


# Save to CSV
df.to_csv('Data/jeju_air.csv',index=False)

In [3]:
# Sanity check
df.head()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
0,jeju air,V Gallera,2020-02-12,✅ Trip Verified | Clark to Seoul. Smooth flig...,Family Leisure,Economy Class,8,4,5,NaN,NaN,5,4,yes
1,jeju air,N Fan,2020-01-21,✅ Trip Verified | Jeju to Hong Kong. The airp...,Solo Leisure,Economy Class,1,1,5,NaN,NaN,1,1,no
2,jeju air,Mei ling Chan,2019-11-21,✅ Trip Verified | Jeju to Seoul. I realize my...,Solo Leisure,Economy Class,1,3,2,NaN,NaN,1,2,no
3,jeju air,A Linton,2019-11-17,Not Verified | My wife was flying in from the...,Family Leisure,Economy Class,1,NaN,NaN,NaN,NaN,1,1,no
4,jeju air,Charlie Norton,2019-10-02,Not Verified | Osaka Kansai to Seoul. Very goo...,Family Leisure,Economy Class,7,2,4,3,NaN,4,3,yes


In [4]:
df.tail()

,airline_name,author,date,content,type_traveller,cabin_flown,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_money_rating,recommended
15,jeju air,Michael Hollifield,2017-08-08,✅ Verified Review | Flew Jeju Air on the quick...,Solo Leisure,Economy Class,9,4,5,NaN,NaN,5,5,yes
16,jeju air,Dmitry Artemyev,2017-05-20,✅ Verified Review | Flew Jeju Air from Seoul ...,Solo Leisure,Economy Class,4,3,4,2,2,2,2,no
17,jeju air,Michael Hollifield,2016-08-24,My first flight with Jeju Air from Seoul to Ko...,Solo Leisure,Economy Class,8,4,5,NaN,NaN,5,5,yes
18,jeju air,Purna Ananth,2016-06-03,Cheongju to Jeju with Jeju Air. Boeing 737-800...,Business,Economy Class,8,3,4,NaN,NaN,4,4,yes
19,jeju air,Handoko Taniwidjojo,2013-07-09,My wife and I were on flight from GMP-CJU and ...,NaN,Economy Class,9,3,4,3,2,NaN,5,yes
